# Ensemble Learning

## Initial Imports

In [88]:
import warnings
warnings.filterwarnings('ignore')

In [89]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

## Read the CSV and Perform Basic Data Cleaning

In [90]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
loan_df = pd.read_csv(file_path)

# Preview the data
loan_df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [151]:
loan_df['home_ownership'] = loan_df['home_ownership'].replace(['OWN'], 0)
loan_df['home_ownership'] = loan_df['home_ownership'].replace(['MORTGAGE'], 1)
loan_df['home_ownership'] = loan_df['home_ownership'].replace(['RENT'], 2)
loan_df['home_ownership'] = loan_df['home_ownership'].replace(['ANY'], 3)
loan_df['verification_status'] = loan_df['verification_status'].replace(['Verified'], 0)
loan_df['verification_status'] = loan_df['verification_status'].replace(['Source Verified'], 1)
loan_df['verification_status'] = loan_df['verification_status'].replace(['Not Verified'], 2)
loan_df['issue_d'] = loan_df['issue_d'].replace(['Jan-2019'], 1.2019)
loan_df['issue_d'] = loan_df['issue_d'].replace(['Feb-2019'], 2.2019)
loan_df['issue_d'] = loan_df['issue_d'].replace(['Mar-2019'], 3.2019)
loan_df['initial_list_status'] = loan_df['initial_list_status'].replace(['w'], 0)
loan_df['initial_list_status'] = loan_df['initial_list_status'].replace(['f'], 1)
loan_df['next_pymnt_d'] = loan_df['next_pymnt_d'].replace(['Apr-2019'], 4.2019)
loan_df['next_pymnt_d'] = loan_df['next_pymnt_d'].replace(['May-2019'], 5.2019)
loan_df['application_type'] = loan_df['application_type'].replace(['Individual'], 0)
loan_df['application_type'] = loan_df['application_type'].replace(['Joint App'], 1)
loan_df['hardship_flag'] = loan_df['hardship_flag'].replace(['N'], 0)
loan_df['debt_settlement_flag'] = loan_df['debt_settlement_flag'].replace(['N'], 0)
loan_df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,2,66000.0,1,3.2019,low_risk,0,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,0,0
1,25000.0,0.2000,929.09,1,105000.0,0,3.2019,low_risk,0,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,0,0
2,20000.0,0.2000,529.88,1,56000.0,0,3.2019,low_risk,0,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,0,0
3,10000.0,0.1640,353.55,2,92000.0,0,3.2019,low_risk,0,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,0,0
4,22000.0,0.1474,520.39,1,52000.0,2,3.2019,low_risk,0,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,0,0


## Split the Data into Training and Testing

In [174]:
# Create features
x_cols = [i for i in loan_df.columns if i not in ("loan_status")]
X = loan_df[x_cols]

# Create target
y = loan_df["loan_status"]

In [175]:
X.describe()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,pymnt_plan,dti,delinq_2yrs,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
count,68817.000000,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,6.881700e+04,68817.0,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.0,6.881700e+04,6.881700e+04,68817.000000,6.881700e+04,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,1.269483,8.821371e+04,1.330006,3.201900e+00,0.0,21.778153,0.217766,...,95.057627,30.626217,0.125972,0.0,2.100332e+05,6.133843e+04,29734.128558,5.572240e+04,0.0,0.0
std,10277.348590,0.048130,288.062432,0.654930,1.155800e+05,0.719105,5.873122e-12,0.0,20.199244,0.718367,...,8.326426,33.631463,0.336732,0.0,1.928088e+05,5.738798e+04,26795.394232,5.095845e+04,0.0,0.0
min,1000.000000,0.060000,30.890000,0.000000,4.000000e+01,0.000000,3.201900e+00,0.0,0.000000,0.000000,...,20.000000,0.000000,0.000000,0.0,3.600000e+03,2.350000e+02,100.000000,1.270000e+02,0.0,0.0
25%,9000.000000,0.088100,265.730000,1.000000,5.000000e+04,1.000000,3.201900e+00,0.0,13.890000,0.000000,...,93.000000,0.000000,0.000000,0.0,6.697700e+04,2.650300e+04,11600.000000,2.288000e+04,0.0,0.0
50%,15000.000000,0.118000,404.560000,1.000000,7.300000e+04,1.000000,3.201900e+00,0.0,19.760000,0.000000,...,100.000000,20.000000,0.000000,0.0,1.467100e+05,4.535700e+04,22100.000000,4.200000e+04,0.0,0.0
75%,24000.000000,0.155700,648.100000,2.000000,1.040000e+05,2.000000,3.201900e+00,0.0,26.660000,0.000000,...,100.000000,50.000000,0.000000,0.0,3.036400e+05,7.657000e+04,39300.000000,7.249900e+04,0.0,0.0
max,40000.000000,0.308400,1676.230000,3.000000,8.797500e+06,2.000000,3.201900e+00,0.0,999.000000,18.000000,...,100.000000,100.000000,4.000000,0.0,3.292782e+06,1.295455e+06,509400.000000,1.426964e+06,0.0,0.0


In [176]:
# Check the balance of target values
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [177]:
# Split the X and y into X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [178]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [179]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_train)

In [180]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [212]:
# Resample the training data with the BalancedRandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
random_forest_model = RandomForestClassifier(n_estimators=100, random_state=78)
random_forest_model = random_forest_model.fit(X_train_scaled, y_train)
random_forest_predictions = random_forest_model.predict(X_test_scaled)

In [213]:
# Calculated the balanced accuracy score
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
random_forest_accuracy_score = accuracy_score(y_test, random_forest_predictions)
random_forest_accuracy_score

0.995640802092415

In [214]:
# Display the confusion matrix
random_forest_confusion_matrix = confusion_matrix(y_test, random_forest_predictions)
random_forest_confusion_matrix_df = pd.DataFrame(
    random_forest_confusion_matrix, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)
random_forest_confusion_matrix_df

,Predicted 0,Predicted 1
Actual 0,34,67
Actual 1,8,17096


In [215]:
# Print the imbalanced classification report
print(classification_report(y_test, random_forest_predictions))

              precision    recall  f1-score   support

   high_risk       0.81      0.34      0.48       101
    low_risk       1.00      1.00      1.00     17104

    accuracy                           1.00     17205
   macro avg       0.90      0.67      0.74     17205
weighted avg       1.00      1.00      0.99     17205



In [216]:
# List the features sorted in descending order by feature importance
importances = rf_model.feature_importances_
importances_sorted = sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)
importances_sorted[:10]

[(0.06483941921864443, 'total_rec_int'),
 (0.06482876589451443, 'total_rec_prncp'),
 (0.06325939001107997, 'last_pymnt_amnt'),
 (0.06086006377489545, 'total_pymnt_inv'),
 (0.05854900042165245, 'total_pymnt'),
 (0.01856478254796249, 'dti'),
 (0.01810978289219453, 'max_bal_bc'),
 (0.017853921398576257, 'out_prncp'),
 (0.01762156027944049, 'mo_sin_old_rev_tl_op'),
 (0.017109686098040387, 'out_prncp_inv')]

### Easy Ensemble Classifier

In [207]:
# Train the Classifier
from imblearn.ensemble import EasyEnsembleClassifier 
easy_ensemble_model = EasyEnsembleClassifier(n_estimators=100, random_state=78)
easy_ensemble_model = easy_ensemble_model.fit(X_train_scaled, y_train)
easy_ensemble_predictions = easy_ensemble_model.predict(X_test_scaled)

ImportError: cannot import name 'EasyEnsembleClassifier' from 'sklearn.ensemble' (/Users/mrose/opt/anaconda3/envs/alpacaenv/lib/python3.7/site-packages/sklearn/ensemble/__init__.py)

In [208]:
# Calculated the balanced accuracy score
easy_ensemble_accuracy_score = accuracy_score(y_test, easy_ensemble_predictions)
easy_ensemble_accuracy_score

NameError: name 'easy_ensemble_predictions' is not defined

In [210]:
# Display the confusion matrix
easy_ensemble_confusion_matrix = confusion_matrix(y_test, easy_ensemble_predictions)
easy_ensemble_confusion_matrix_df = pd.DataFrame(
    easy_ensemble_confusion_matrix, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)
easy_ensemble_confusion_matrix_df

NameError: name 'easy_ensemble_predictions' is not defined

In [209]:
# Print the imbalanced classification report
print(classification_report(y_test, easy_ensemble_predictions))

NameError: name 'easy_ensemble_predictions' is not defined

### Final Questions

1. Which model had the best balanced accuracy score?

    YOUR ANSWER HERE.

2. Which model had the best recall score?

    YOUR ANSWER HERE.

3. Which model had the best geometric mean score?

    YOUR ANSWER HERE.

4. What are the top three features?

    YOUR ANSWER HERE.